The Turán number of $C^3_5$
==============================

This notebook contains calculations for the Turán number of $C^3_5$

The conjectured optimal construction is two partite with edges AAB and recursion inside B, where $d(A) = (3-\sqrt{3})/2$, as calculated in the overleaf

This only contains the last step: in the colored theory, can the computer prove that Bad <= Missing?

In [2]:
from sage.algebras.flag_algebras import *

In [3]:
###
### This cell is just to set up the theory.
###

from sage.algebras.flag_algebras import *

# These are helper functions, to deal with classical exclusion (not just induced)
def check_containment(smalls, larges):
    """
    Helper function to check is any of the smalls appears in each of the larges.

    INPUT:
    smalls - list of flags, must be from a theory with edges relation
    larges - list of flags, also must be from a theory with edges relation

    OUTPUT:
    list of booleans, i-th element represents if i-th large flag is free from all smalls
    """
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in larges]
    res = []
    for ll in lis:
        good = True
        for ss in sis:
            for _ in ll.isomorphic_substructures_iterator(ss):
                good = False
                break
            if not good:
                break
        res.append(good)
    return res

def check_containment_cert(smalls, large):
    """
    Helper function to check is any of the smalls appears in the large, and if yes,
    returns an injection of the small

    INPUT:
    smalls - list of flags, must be from a theory with edges relation
    large - a flag, also must be from a theory with edges relation

    OUTPUT:
    empty list ([]) if all smalls is avoided, otherwise [small, mapping] telling how to
    inject small into large with mapping
    """
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = IncidenceStructure(large.size(), large.blocks()['edges'])
    for ii, ss in enumerate(sis):
        for xx in lis.isomorphic_substructures_iterator(ss):
            return [xx, smalls[ii]]
    return None

# This is some hack to create the theory for 3-graphs without C5- and K4-
# up to size 7. It is easier to make them as extensions of 6 sized structures
# so this code does that.

# Reset three graphs, so nothing is excluded
TG = ThreeGraphTheory
TG.exclude()

# C5 minus
C5 = TG(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0], [4, 0, 1]])

# flags of size 5
fl5 = TG.generate_flags(5)

# boolean vector indicating each element in fl5 if it has C5m
gs = check_containment([C5], fl5)

# set the excluded structures. k4, k4m and all in fl5 containing C5m
exls = [xx for ii, xx in enumerate(fl5) if not gs[ii]]
TG.exclude(exls)

# check the list of flags with size 5 and 6
fl5 = TG.generate_flags(5)
fl6 = TG.generate_flags(6)


# number of flags with size 5 and 6
# should be 26 835
print(len(fl5), len(fl6))


# This is code to create colored theories
# This is a default code for all color partition
def _identifyCT(k, order_partition, n, ftype_points, **kwargs):
    is_graph = (k==2)
    color_number = sum(len(xx) for xx in order_partition)
    edges = kwargs["edges"]
    Cs = [[cx[0] for cx in kwargs["C{}".format(ii)]] for ii in range(color_number)]
    g_parts = [[ii] for ii in ftype_points] + \
              [[ii for ii in range(n) if ii not in ftype_points]]
    ppadd = 0 if is_graph else len(edges)
    g_verts = list(range(n+ppadd+color_number))
    g_parts.append(list(range(n, n+ppadd)))

    g_parts += [[n+ppadd+ii for ii in partition_j] for partition_j in order_partition]
    
    if is_graph:
        g_edges = list(edges)
        for ii in range(color_number):
            g_edges += [(xx, n+ii) for xx in Cs[ii]]
    else:
        g_edges = [(i+n,x) for i,b in enumerate(edges) for x in b]
        for ii in range(color_number):
            g_edges += [(xx, n+len(edges)+ii) for xx in Cs[ii]]
    g = Graph([g_verts, g_edges], format='vertices_and_edges')
    blocks = tuple(g.canonical_label(partition=g_parts).edges(labels=None, sort=True))
    ftype_points = tuple(range(len(ftype_points)))
    return (n, ftype_points, blocks)

# This is also a default code for all color partition
def _generateCT(base_theory, k, order_partition, n):
    color_number = sum(len(xx) for xx in order_partition)
    BT = base_theory
    for xx in BT.generate_flags(n):
        unique = []
        edges = xx.blocks()['edges']
        
        for yy in itertools.product(range(color_number), repeat=int(n)):
            yy = list(yy)
            Cs = {"C{}".format(cc):[[ii] for ii, oo in enumerate(yy) if oo==cc] for cc in range(color_number)}
            iden = _identifyCT(k==2, order_partition, n, [], edges=edges, **Cs)
            if iden not in unique:
                unique.append(iden)
                Cs["edges"] = edges
                yield Cs

# To make the default codes work for this specific case:
# The generator:
# Colors the elements of TGp (3-graphs without C5- and K4-), works on 3-uniform structures
# and the colors 0, 1, 2 are interchangeable (otherwise it would say [[0], [1], [2]]
def generate_colored(n):
    return _generateCT(TG, 3, [[0], [1]], n)

# Same for the identifier. Colors are interchangeable.
def identify_colored(n, ftype_points, edges, C0, C1):
    return _identifyCT(3, [[0], [1]], n, ftype_points, edges=edges, C0=C0, C1=C1)

# CTG is the colored variant of TG (No C5 in ThreeGraphTheory)
CTG = CombinatorialTheory("ColoredNoC5", generate_colored, identify_colored, edges=3, C0=1, C1=1)

# sanity check, the number of flags with size 4, 5, 6
#should be 35 430 40547
print(len(CTG.generate_flags(4)), len(CTG.generate_flags(5)), len(CTG.generate_flags(6)))

26 835
35 430 40547


In [6]:
# edge with correct colors
C = CTG(3, edges=[[0, 1, 2]], C0=[[0], [1]], C1=[[2]])
#missing edge with good colors
M = CTG(3, edges=[], C0=[[0], [1]], C1=[[2]])

# pointed edge with correct colors, point from color 0
Cp0 = CTG(3, edges=[[0, 1, 2]], C0=[[0], [1]], C1=[[2]], ftype=[0])
# pointed edge with correct colors, point from color 1
Cp1 = CTG(3, edges=[[0, 1, 2]], C0=[[0], [1]], C1=[[2]], ftype=[2])

# edge with bad colors all in color 0
B000 = CTG(3, edges=[[0, 1, 2]], C0=[[0], [1], [2]], C1=[])

# edge with bad colors, looking the wrong way
B011 = CTG(3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1], [2]])

# pointed edge with bad colors, point from moving 1 -> 0
Bp0 = CTG(3, edges=[[0, 1, 2]], C0=[[0], [1], [2]], C1=[], ftype=[0])

# pointed edge with bad colors, point from moving 0 -> 1
Bp1 = CTG(3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1], [2]], ftype=[2])

# positivity assumptions
# each point, good edges are more than bad edges that would become good after moving the point
# good edge density is larger than some constant (here 0.4413)
colored_assums = [Cp0 - Bp0, Cp1 - Bp1, C - 4413/10000]

# bad is less than missing
max_bmm = CTG.optimize_problem(B000 + B011 - M, 5, maximize=True, positives=colored_assums)
# I get 0.0.005172173040424277, I doubt even with rounding this gets 0
print("\n\n", max_bmm, "\n\n")

Ftypes constructed in 0.47s
Block sizes done in 0.17s
Block sizes are [6, 6, 16, 16, 16, 16, 16, 16, 16, 16, -430, -17]
Calculating product matrices for 10 ftypes and 430 structures
Ftype on 3 points with edges=[[0, 1, 2]], C0=[], C1=[[0], [1], [2]] is complete: : 10it [00:00, 106.16it/s]
Table calculation done in 0.11s
Target and constraint calculation done in 0.13s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -3.6646092e+01 Ad: 4.26e-01 Dobj:  1.0619723e+01 
Iter:  2 Ap: 1.00e+00 Pobj: -3.8621859e+01 Ad: 9.36e-01 Dobj:  2.3775501e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -4.0744233e+01 Ad: 8.59e-01 Dobj:  4.8960471e-02 
Iter:  4 Ap: 1.00e+00 Pobj: -4.2434269e+01 Ad: 6.56e-01 Dobj: -2.1664701e-02 
Iter:  5 Ap: 9.10e-01 Pobj: -3.1435905e+01 Ad: 5.45e-01 Dobj: -7.1274989e-02 
Iter:  6 Ap: 9.00e-01 Pobj: -1.9482396e+01 Ad: 6.66e-01 Dobj: -5.3354177e-02 
Iter:  7 Ap: 1.00e+00 Pobj: -1.1959406e+01 Ad: 6.76e-01 Dobj: -3.0172123